# Dunebuggy
---
A lightweight (unofficial) Python SDK for Dune.xyz

## Installation

```sh
pip install dunebuggy
```

## Getting started

### Retrieving a public query

To retrieve a query, all we'll need is the ```query_id``` for the public query we're interested in. In the below example we can take a look at the popular ["Custom NFT Floor Tracker" query by @smaroo](https://dune.xyz/queries/83579) (The ```query_id``` below can be found in the URL).

In [1]:
from dunebuggy import Dune

dune = Dune()
query = dune.fetch_query(83579)
query.df.head()


,Floor Price Ξ,Time Interval,Volume Ξ
0,0.0700,2021-06-30T00:00:00+00:00,436.250420
1,0.0670,2021-07-01T00:00:00+00:00,234.422398
2,0.0553,2021-07-02T00:00:00+00:00,126.205068
3,0.0530,2021-07-03T00:00:00+00:00,41.314672
4,0.0490,2021-07-04T00:00:00+00:00,40.311550


We can also take a look at some basic information about the returned query with ```query.info```

In [ ]:
query.info

Some queries in Dune are "parameterized", meaning that the author exposes certain variables for the user to enter custom values. The example query (83579) happens to be parameterized, we can verify this by inspecting ```query.parameters```

In [ ]:
query.parameters

If you'd like to run this query with your own custom parameters, all we'll need to do is take the parameters from from the initial query, change the values to what we want, and re-fetch the query. You can also create a fresh set of parameters by importing ```QueryParameter``` from ```dunebuggy.models.query``` and adding the values to the new object.

Below we are replacing the old NFT contract address param with a new one ([the contract address for BAYC](https://etherscan.io/address/0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d))

In [ ]:
old_params = query.parameters

# Replacing with contract address for BAYC
old_params[0].value = 'xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D'
new_params = old_params
print(new_params)
custom_query = dune.fetch_query(83579, parameters=new_params)

custom_query.info

Note that the ```result_id``` and ```job_id``` here are different, this is because we ran the query with our changed params

In [ ]:
custom_query.parameters

In [ ]:
custom_query.df.head()

### Creating a new query

Dunebuggy also allows you to create a new using an existing Dune.xyz account.You'll just need to pass in your username/password into the ```Dune``` object in order to login. After logging in, you should be able to retrieve your ```user_id```

In [ ]:
import os 

username = os.environ.get('DUNE_USERNAME')
password = os.environ.get('DUNE_PASSWORD')

dune = Dune(username=username, password=password)
dune.user_id

We'll now need to construct a Dune SQL query. We can do this in two ways. The first being just creating a raw string SQL query like below

In [ ]:
query_string = "select * from ethereum.transactions\nLIMIT 100\n"

Or, if we wanna get fancy, we could use the fantastic PyPika library to construct one in an ORM style

In [ ]:
from pypika import Database, Query

ethereum = Database('ethereum')
q = Query.from_(ethereum.transactions).select('*').limit(100)
query_string = q.get_sql(quote_char=None)
query_string

Dune requires us to specify an integer code (```Id```) for each of their support blockchain datasets. The currently supported datasets are the following:

| Blockchain Dataset | Id |
|--------------------|----|
| ETHEREUM           | 4  |
| XDAI               | 6  |
| POLYGON            | 7  |
| OPTIMISM_1         | 8  |
| OPTIMISM_2         | 10 |
| BINANCE            | 9  |
| SOLANA             | 1  |

We can access these integer codes via the ```DatasetId``` enum. To create a query now, all we need to do is pass in a ```name```, ```query_string``` and ```dataset_id```

In [ ]:
from dunebuggy.models.constants import DatasetId
created_query = dune.create_query("My Query's Name", query_string, DatasetId.ETHEREUM)                          
                       

In [ ]:
created_query.df.head()

### Saving to CSV

To save a query to a CSV, we can take advantage of the ```to_csv``` method on our ```df```

In [ ]:
created_query.df.to_csv('my_test_data.csv')